- Classes in torch.nn, they are parts for building neural netwoek.
- Those classes is called module.
- All module inherit `nn.Module`

In [1]:
%matplotlib inline
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device}')

Using cuda


- 1. inherit `nn.Module`
- 2. define your network model
- 3. initialize netwoek in `__init__`
- 4. All module that inherit `nn.Module` must have `forward()`

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # 28x28 is the input size, 512 is the output size
            nn.ReLU(), # 512 is input size, 512 is output size
            nn.Linear(512, 512), # 512 is input size, 512 is output size
            nn.ReLU(), # 512 is input size, 512 is output size
            nn.Linear(512, 10), # 512 is input size, 10 is output size
            nn.ReLU(), # 10 is input size, 10 is output size
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([1], device='cuda:0')


# model layer

In [9]:
input_image = torch.rand(3, 28, 28)
print(input_image.shape)

torch.Size([3, 28, 28])


`nn.Flatten` 0 dim is not flatten, after 1 dims are flatten

In [10]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.shape)

torch.Size([3, 784])


`nn.Linear` xW+b

In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.shape)

torch.Size([3, 20])


``nn.ReLU`

In [13]:
print(f'before ReLU: {hidden1}')
hidden1 = nn.ReLU()(hidden1)
print(f'after ReLU: {hidden1}')

before ReLU: tensor([[-0.1650, -0.1680,  0.7445, -0.4745, -0.1140,  0.5935,  0.0063,  0.7916,
         -0.3127,  0.2021, -0.1078,  0.1441, -0.3508,  0.3155, -0.3275,  0.0977,
         -0.2250,  0.0784, -0.4256,  0.1897],
        [-0.0969, -0.3261,  0.8233, -0.4090, -0.0905,  0.5619,  0.2942,  0.5209,
         -0.3535,  0.2897,  0.1022, -0.0994, -0.1151,  0.2917, -0.1245, -0.1326,
         -0.2376, -0.2648, -0.3912, -0.1481],
        [-0.0456, -0.3122,  0.4454, -0.1934,  0.2133,  0.4002, -0.0619,  0.3943,
         -0.3434,  0.3976,  0.1857, -0.0579, -0.0502,  0.1374, -0.4048, -0.2739,
         -0.3232, -0.2021, -0.6361, -0.2934]], grad_fn=<AddmmBackward0>)
after ReLU: tensor([[0.0000, 0.0000, 0.7445, 0.0000, 0.0000, 0.5935, 0.0063, 0.7916, 0.0000,
         0.2021, 0.0000, 0.1441, 0.0000, 0.3155, 0.0000, 0.0977, 0.0000, 0.0784,
         0.0000, 0.1897],
        [0.0000, 0.0000, 0.8233, 0.0000, 0.0000, 0.5619, 0.2942, 0.5209, 0.0000,
         0.2897, 0.1022, 0.0000, 0.0000, 0.2917, 0.0000

`nn.Sequential`

In [14]:
seq_modules = nn.Sequential(
    flatten, 
    layer1,
    nn.ReLU(),
    nn.Linear(in_features=20, out_features=10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [15]:
logits

tensor([[ 0.1758, -0.1955, -0.2526,  0.1106,  0.3715,  0.0722,  0.0688, -0.1547,
         -0.0677, -0.0734],
        [ 0.0894, -0.2923, -0.1295,  0.2382,  0.3556,  0.2425,  0.0712, -0.2163,
         -0.1415, -0.1218],
        [ 0.1417, -0.2062, -0.1452,  0.1804,  0.2750,  0.2320,  0.0111, -0.1711,
         -0.0038, -0.1244]], grad_fn=<AddmmBackward0>)

`nn.Softmax`

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
pred_probab

tensor([[0.1166, 0.0804, 0.0760, 0.1092, 0.1418, 0.1051, 0.1048, 0.0838, 0.0914,
         0.0909],
        [0.1059, 0.0723, 0.0851, 0.1229, 0.1383, 0.1235, 0.1040, 0.0780, 0.0841,
         0.0858],
        [0.1114, 0.0787, 0.0836, 0.1158, 0.1273, 0.1220, 0.0978, 0.0815, 0.0964,
         0.0854]], grad_fn=<SoftmaxBackward0>)

# model parameters

In [18]:
print(f'Model structure: {model}')

for name, param in model.named_parameters():
    print(f'Layer: {name}, Size: {param.size()} | Values: {param[:2]} \n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)
Layer: linear_relu_stack.0.weight, Size: torch.Size([512, 784]) | Values: tensor([[ 0.0126,  0.0237,  0.0057,  ...,  0.0126, -0.0026,  0.0058],
        [ 0.0162, -0.0185, -0.0244,  ..., -0.0026, -0.0047,  0.0099]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias, Size: torch.Size([512]) | Values: tensor([ 0.0039, -0.0012], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight, Size: torch.Size([512, 512]) | Values: tensor([[ 0.0400,  0.0344,  0.0303,  ...,  0.0270,  0.0321, -0.0034],
        [-0.0114,  0.0212, -0.0089,  ...,  0.0167, -0.0110, -0.0143]],
       device='cuda:0', gra